In [8]:
from functools import partial
import sys
sys.path.append('../src')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import KBio

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
mse = mean_squared_error

from matplotlib.gridspec import GridSpec

In [9]:
Gauss1 = KBio.Gaussian(sigma=1e-2)
Gauss2 = KBio.Gaussian(sigma=2e-1)
#Gauss2 = KBio.Gaussian(sigma=5e-2)
lin = KBio.Polynomial(degree=1, c=0)
poly_quad = KBio.Polynomial(degree=2, c=0.01)
poly_cubic = KBio.Polynomial(degree=3, c=0.01)
poly_quartic = KBio.Polynomial(degree=4, c=0.01)
poly_quintic = KBio.Polynomial(degree=5, c=0.01)

In [16]:
#operator_kernels = [Gauss2, lin, poly_quad, poly_cubic, poly_quartic, poly_quintic]
operator_kernels = [Gauss2]

In [17]:
# Simulation parameters
beta = 4e-1
gamma = 1e-1
T_final = 3e1
dt = 1e-2

# declare the simulator
sim = KBio.SIS_sim(dt=dt, T_final=T_final, beta=beta, gamma=gamma, I0 = 10, S0 = 90)

# number of samples for the training and test sets
n_samples_oscillatory = 12 #20  

# Use consistent forcings for standardization between trials
#amplitudes_list = np.linspace(0.1, 0.5, 5)
amplitudes_list = np.linspace(0.1, 0.2, 2)
amplitudes_list = np.sort(amplitudes_list)

np.random.seed(1947)
shifts = np.random.uniform(0, np.pi, n_samples_oscillatory)
# frequencies = np.random.uniform(1, 6, n_samples_oscillatory)
frequencies = [np.random.normal(i+1, 0.1, n_samples_oscillatory // 5) for i in range(5)]
frequencies = np.concatenate(frequencies)

# Derivatives of the forcing function to consider
alphas_list = [[0], [1], [2]]

In [18]:
def oscillatory_forcing(A, omega, phi, t):
    return A*np.sin(omega + phi*t) * np.minimum(1, t/10)

def constant_forcing(A, t):
    return A * np.minimum(1, t/10)

In [19]:
# feature functions
feature_functions = []

def ident(index, x, u, u_deriv_list):
    """ Identity feature function for the $index$-th derivative of the function"""
    if np.abs(int(index) - index) > 1e-10:
        raise ValueError('Only the first derivative is available')
    if index == 0:
        return u
    else:
        return u_deriv_list[index]
# Get the first 2 derivatives and the function itself
for i in range(1,3):
    feature_functions.append(partial(ident, i))

# quadratic feature function
def quadratic(index, x, u, u_deriv_list):
    """ Quadratic feature function for the $index$-th derivative of the function

    The feature function is $u^2$ for the function itself and $2u u'$ for the first derivative.
    """

    if index == 0:
        return u**2
    elif index == 1:
        return 2*u*u_deriv_list[index]
    else:
        raise ValueError('Higher derivatives are not implemented')

feature_functions.append(partial(quadratic, 0))

In [20]:
training_data = {}
for amplitude_ref in amplitudes_list:

    amplitudes = np.full(n_samples_oscillatory,amplitude_ref)

    ## Training data
    forcing_functions = [partial(oscillatory_forcing, amplitude, shift, frequency) for amplitude, shift, 
                         frequency in zip(amplitudes, shifts, frequencies)]
    grids_oscillatory = [KBio.rectangular_grid([0], [30], [301]) for _ in range(n_samples_oscillatory)]
    for grid, amplitude, shift, frequency, fn in zip(grids_oscillatory, amplitudes, shifts, frequencies, forcing_functions):
        sim(grid, forcing = fn)

    training_data[amplitude_ref] = {'grids': grids_oscillatory, 'forcing_functions': forcing_functions}

In [21]:
lambda_dict = {}
lambda_errors = {}
CV_mse = {}
lambda_opts = np.logspace(-2, 0, 3)
#lambda_opts = np.logspace(-9, 0, 21)

In [24]:

kf = KFold(n_splits=4, shuffle=True, random_state=1947)
for amplitude_ref in amplitudes_list:

    grids_train = training_data[amplitude_ref]["grids"]

    # standardize the data
    x_data_train = np.stack([grid.grid_tensors[0] for grid in grids_train])
    x_mean = np.mean(grids_train[0].grid_tensors[0])
    x_std = np.std(grids_train[0].grid_tensors[0])
    x_data_train = (x_data_train - x_mean) / x_std

    y_data_train = np.stack([grid.grid_tensors_values for grid in grids_train])
    f_data_train = np.stack([grid.grid_tensors_forcing for grid in grids_train])

    for i, (train_index, test_index) in enumerate(kf.split(x_data_train)):
        #mse_list = []
        for kernel in operator_kernels:
            mse_list = []
            for lambda_opt in lambda_opts:
                z, kdList, u_smoothed_train = KBio.kernel_smoothing(Gauss1, x_grid=x_data_train[train_index], 
                                                            u_data=y_data_train[train_index], f_data=f_data_train[train_index], 
                                                            alpha_list=alphas_list, nugget=1e-8)
                x_grid_list = grid.grid_list

                ## Assembling the features for the training data
                F0_train = KBio.assemble_features(x_grid_list=x_grid_list, u_smoothed=u_smoothed_train[0], 
                                            multi_derivatives=u_smoothed_train, function_list=feature_functions,flatten=True)
                F0_train_mean = np.mean(F0_train, axis=0)
                F0_train_std = np.std(F0_train, axis=0)
                F0_train = np.divide((F0_train - F0_train_mean), F0_train_std, where = (F0_train_std != 0))

                f_train_flat = f_data_train[train_index].reshape(-1, 1)
                ## Learning the predictor
                predictor, weight_vector = KBio.learn_DE_form(kernel=Gauss2, s_features=F0_train, f_labels=f_train_flat, 
                                                            nugget=lambda_opt)

                ## Smoothing the test data 
                z_test, kdList_test, u_smoothed_test = KBio.kernel_smoothing(Gauss1, x_grid=x_data_train[test_index], u_data=y_data_train[test_index], 
                                                                            f_data=f_data_train[test_index], alpha_list=alphas_list, nugget=1e-8)
                F0_test = KBio.assemble_features(x_grid_list=x_grid_list, u_smoothed=u_smoothed_test[0], multi_derivatives=u_smoothed_test, 
                                                function_list=feature_functions, flatten=False)
                F0_test = np.divide((F0_test - F0_train_mean), F0_train_std, where = (F0_train_std != 0))

                #f_test_flat = f_data_train[test_index].reshape(-1, 1)
                preds = []
                for index in range(F0_test.shape[0]):
                    _, p = predictor(F0_test[index,:])
                    preds.append(p)
                preds = np.hstack(preds)
                mse_list.append(mse(f_data_train[test_index].T, preds))

            CV_mse[(amplitude_ref, i, kernel)] = mse_list

lambda_dict[(amplitude_ref, kernel)] = lambda_opts[np.argmin(np.mean([CV_mse[(amplitude_ref, i, kernel)] for i in range(4)], axis=0))]
lambda_errors[(amplitude_ref, kernel)] = np.mean([CV_mse[(amplitude_ref, i, kernel)] for i in range(4)], axis=0)

      

ValueError: all input arrays must have the same shape